## Chassis Example Notebooks
Welcome to the examples section for [Chassis](https://chassis.ml), which contains notebooks that auto-containerize models built using the most common machine learning (ML) frameworks. 

#### What is Chassis?
Chassis allows you to automatically create a Docker container from your model code and push that container image to a Docker registry. All you need is your model loaded into memory and a few lines of Chassis code! Our example bank is here to provide reference examples for many common ML frameworks.  

Can't find the framework you are looking for or need help? Fork this repository and open a PR, or list the desired framework in a new issue. We're always interested in growing this example bank! 

The primary maintainers of Chassis also actively monitor our [Discord Server](https://discord.gg/cHpzY9yCcM), so feel free to join and ask any questions you might have. We'll be there to respond and help out promptly.

In [1]:
import time
import chassisml
import pickle
import cv2
import torch
import getpass
import numpy as np
import torchvision.models as models
from torchvision import transforms

## Enter credentials
Dockerhub creds and Modzy API Key

In [ ]:
dockerhub_user = getpass.getpass('docker hub username')
dockerhub_pass = getpass.getpass('docker hub password')

## Prepare model, labels, and data transform
Initialize anything here that should persist across inference runs

In [3]:
model = models.detection.fasterrcnn_mobilenet_v3_large_fpn(pretrained=True)
model.eval()

COCO_INSTANCE_CATEGORY_NAMES = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A',
    'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
    'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table',
    'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book',
    'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]

transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
])       

device = 'cpu'

def preprocess(input_bytes):
    decoded = cv2.imdecode(np.frombuffer(input_bytes, np.uint8), -1)
    img_t = transform(decoded)
    batch_t = torch.unsqueeze(img_t, 0).to(device)
    return batch_t

def postprocess(num_detections, predictions):
    inference_result = {
        "detections": [
            {
                "xMin": predictions["boxes"][i].detach().cpu().numpy().tolist()[0],
                "xMax": predictions["boxes"][i].detach().cpu().numpy().tolist()[2],
                "yMin": predictions["boxes"][i].detach().cpu().numpy().tolist()[1],
                "yMax": predictions["boxes"][i].detach().cpu().numpy().tolist()[3],
                "class": labels[predictions["labels"][i].detach().cpu().item()],
                "classProbability": predictions["scores"][i].detach().cpu().item(),
            } for i in range(num_detections)
        ]
    }

    structured_output = {
        "data": {
            "result": inference_result,
            "explanation": None,
            "drift": None,
        }
    }    
    return structured_output

## Write process function

* Must take bytes as input
* Preprocess bytes, run inference, postprocess model output, return results

In [5]:
def process(input_bytes):
    
    # preprocess
    batch_t = preprocess(input_bytes)
    
    # run inference
    predictions = model(batch_t)[0]
    num_detections = len(predictions["boxes"])
    
    # postprocess
    structured_output = postprocess(num_detections, predictions)
    
    return structured_output

## Initialize Chassis Client
We'll use this to interact with the Chassis service

In [6]:
chassis_client = chassisml.ChassisClient("https://chassis.app.modzy.com")

## Create and test Chassis model
* Requires `process_fn` defined above

In [ ]:
# create Chassis model
chassis_model = chassis_client.create_model(process_fn=process)

# test Chassis model (can pass filepath, bufferedreader, bytes, or text here):
sample_filepath = './data/airplane.jpg'
results = chassis_model.test(sample_filepath)
print(results)

## Publish model to Docker
Need to provide model name, model version, and Dockerhub credentials

In [ ]:
MODEL_NAME = "PyTorch Faster R-CNN Object Detection"
start_time = time.time()
response = chassis_model.publish(
    model_name=MODEL_NAME,
    model_version="0.0.2",
    registry_user=dockerhub_user,
    registry_pass=dockerhub_pass
)

In [13]:
job_id = response.get('job_id')
final_status = chassis_client.block_until_complete(job_id)
end_time = time.time()
if final_status['status']['succeeded'] == 1:
    print("Job Completed in {} minutes.\n\nView your new container image here: https://hub.docker.com/repository/docker/{}/{}".format(round((end_time-start_time)/60, 5), dockerhub_user, "-".join(MODEL_NAME.lower().split(" "))))
else:
    print("Job Failed. See logs below:\n\n{}".format(final_status['logs']))